Crop into QLD

In [ ]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray

# Load the shapefile for Queensland
shapefile_path = r"Data\QLD Bio Regions\QLD_BioRegion_Shape_Files\Biogeographic_Regions.shp"
queensland_shape = gpd.read_file(shapefile_path)

# Ensure the shapefile has the same CRS as the NetCDF file
queensland_shape = queensland_shape.to_crs("EPSG:4326")  # Update CRS if needed

# Folder containing .nc files
nc_folder = r"Data - Modelling\Annual_DB\1. AUS\Rainfall"  # Replace with your folder path

# Get a list of all .nc files in the folder
nc_files = [f for f in os.listdir(nc_folder) if f.endswith(".nc")]

# Loop through each .nc file and crop it
for nc_file in nc_files:
    # Full path to the .nc file
    nc_file_path = os.path.join(nc_folder, nc_file)
    
    # Open the NetCDF file
    ds = xr.open_dataset(nc_file_path)
    
    # Convert to a GeoTIFF-compatible format
    ds = ds.rio.write_crs("EPSG:4326", inplace=True)
    
    # Clip the dataset using the Queensland shapefile
    clipped = ds.rio.clip(queensland_shape.geometry, queensland_shape.crs, drop=True)
    
    # Save the cropped dataset to a new NetCDF file
    output_file = os.path.join(r"Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall", f"cropped_{nc_file}")
    clipped.to_netcdf(output_file)
    print(f"Cropped file saved as {output_file}")

Crop into Region wise

In [ ]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray

# Load the shapefile for Queensland
shapefile_path = r"Data\QLD Bio Regions\QLD_BioRegion_Shape_Files\Biogeographic_Regions.shp"
queensland_shape = gpd.read_file(shapefile_path)

# Ensure the shapefile has the same CRS as the NetCDF file
queensland_shape = queensland_shape.to_crs("EPSG:4326")  # Update CRS if needed

# Folder containing .nc files
nc_folder = r"Data - Modelling\Annual_DB\1. AUS\Rainfall"  # Replace with your folder path

# Get a list of all .nc files in the folder
nc_files = [f for f in os.listdir(nc_folder) if f.endswith(".nc")]

# Loop through each .nc file
for nc_file in nc_files:
    # Full path to the .nc file
    nc_file_path = os.path.join(nc_folder, nc_file)
    
    # Open the NetCDF file
    ds = xr.open_dataset(nc_file_path)
    
    # Convert to a GeoTIFF-compatible format
    ds = ds.rio.write_crs("EPSG:4326", inplace=True)
    
    # Loop through each region in the Queensland shapefile
    for index, region in queensland_shape.iterrows():
        # Get the region ID (q_reg)
        region_id = region['q_reg']
        
        # Clip the dataset using the geometry of the current region
        region_clipped = ds.rio.clip([region.geometry], queensland_shape.crs, drop=True)
        
        # Save the cropped dataset to a new NetCDF file with the region ID in the filename
        output_file = os.path.join(r"Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall", f"cropped_{region_id}_{nc_file}")
        region_clipped.to_netcdf(output_file)
        print(f"Cropped file for region {region_id} saved as {output_file}")

In [6]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray

# === Paths ===
shapefile_path = r"Data\QLD Bio Regions\QLD_BioRegion_Shape_Files\Biogeographic_Regions.shp"
input_nc_folder = r"Data - Modelling\Annual_DB\1. AUS\Max Temp"
output_nc_folder = r"Data - Modelling\Annual_DB\2. QLD_Cropped\Max Temp"

os.makedirs(output_nc_folder, exist_ok=True)

# === Load shapefile ===
queensland_shape = gpd.read_file(shapefile_path)
queensland_shape = queensland_shape.to_crs("EPSG:4326")  # Match CRS to NetCDF

# === List all .nc files ===
nc_files = [f for f in os.listdir(input_nc_folder) if f.endswith(".nc")]

# === Loop through each NetCDF file ===
for nc_file in nc_files:
    nc_path = os.path.join(input_nc_folder, nc_file)
    print(f"\nProcessing file: {nc_file}")
    
    try:
        ds = xr.open_dataset(nc_path)
        ds = ds.rio.write_crs("EPSG:4326", inplace=True)

        # === Loop through each region ===
        for idx, row in queensland_shape.iterrows():
            region_code = row["q_reg"]
            region_name = row["q_reg_name"]
            geometry = [row.geometry]

            try:
                # Clip with region geometry
                clipped = ds.rio.clip(geometry, queensland_shape.crs, drop=True)
                
                # Add region info as metadata
                clipped.attrs["region_code"] = region_code
                clipped.attrs["region_name"] = region_name

                # Output filename
                output_filename = f"{region_code}_cropped_{nc_file}"
                output_path = os.path.join(output_nc_folder, output_filename)

                # Save to NetCDF
                clipped.to_netcdf(output_path)
                print(f"  ✅ Saved: {output_filename}")

            except Exception as region_err:
                print(f"  ⚠️  Skipped region {region_code}: {region_err}")
    
    except Exception as file_err:
        print(f"❌ Failed to process {nc_file}: {file_err}")



Processing file: 2000.max_temp.nc
  ✅ Saved: BRB_cropped_2000.max_temp.nc
  ✅ Saved: CHC_cropped_2000.max_temp.nc
  ✅ Saved: CQC_cropped_2000.max_temp.nc
  ✅ Saved: CYP_cropped_2000.max_temp.nc
  ✅ Saved: DEU_cropped_2000.max_temp.nc
  ✅ Saved: EIU_cropped_2000.max_temp.nc
  ✅ Saved: GUP_cropped_2000.max_temp.nc
  ✅ Saved: MGD_cropped_2000.max_temp.nc
  ✅ Saved: MUL_cropped_2000.max_temp.nc
  ✅ Saved: NET_cropped_2000.max_temp.nc
  ✅ Saved: NWH_cropped_2000.max_temp.nc
  ✅ Saved: SEQ_cropped_2000.max_temp.nc
  ✅ Saved: WET_cropped_2000.max_temp.nc

Processing file: 2001.max_temp.nc
  ✅ Saved: BRB_cropped_2001.max_temp.nc
  ✅ Saved: CHC_cropped_2001.max_temp.nc
  ✅ Saved: CQC_cropped_2001.max_temp.nc
  ✅ Saved: CYP_cropped_2001.max_temp.nc
  ✅ Saved: DEU_cropped_2001.max_temp.nc
  ✅ Saved: EIU_cropped_2001.max_temp.nc
  ✅ Saved: GUP_cropped_2001.max_temp.nc
  ✅ Saved: MGD_cropped_2001.max_temp.nc
  ✅ Saved: MUL_cropped_2001.max_temp.nc
  ✅ Saved: NET_cropped_2001.max_temp.nc
  ✅ Saved:

In [8]:
import os
import json
import xarray as xr
import geopandas as gpd
import rioxarray
import rasterio.features
import numpy as np

# === Paths ===
shapefile_path = r"Data\QLD Bio Regions\QLD_BioRegion_Shape_Files\Biogeographic_Regions.shp"
input_nc_folder = r"Data - Modelling\Annual_DB\1. AUS\Rainfall"
output_nc_folder = r"Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall"
os.makedirs(output_nc_folder, exist_ok=True)

# === Load shapefile and assign integer codes ===
qld_shape = gpd.read_file(shapefile_path).to_crs("EPSG:4326")
region_list = qld_shape["q_reg"].tolist()
region_code_map = {code: idx for idx, code in enumerate(region_list)}
reverse_map = {idx: code for code, idx in region_code_map.items()}  # for JSON output

# === Add integer codes to GeoDataFrame ===
qld_shape["region_id"] = qld_shape["q_reg"].map(region_code_map)

# === List .nc files ===
nc_files = [f for f in os.listdir(input_nc_folder) if f.endswith(".nc")]

# === Process each file ===
for nc_file in nc_files:
    nc_path = os.path.join(input_nc_folder, nc_file)
    print(f"\n📂 Processing: {nc_file}")

    try:
        ds = xr.open_dataset(nc_path)
        ds = ds.rio.write_crs("EPSG:4326", inplace=True)

        # === Clip dataset to Queensland bounding geometry ===
        clipped = ds.rio.clip(qld_shape.geometry, qld_shape.crs, drop=True)

        # === Prepare rasterization ===
        transform = clipped.rio.transform()
        out_shape = (clipped.sizes["lat"], clipped.sizes["lon"])

        shapes = [
            (geom, rid) for geom, rid in zip(qld_shape.geometry, qld_shape.region_id)
        ]
        region_raster = rasterio.features.rasterize(
            shapes=shapes,
            out_shape=out_shape,
            transform=transform,
            fill=-1,
            dtype=np.int16
        )

        # === Add region_code variable to dataset ===
        clipped["region_code"] = (("lat", "lon"), region_raster)

        # === Save output ===
        output_path = os.path.join(output_nc_folder, f"QLD_{nc_file}")
        clipped.to_netcdf(output_path)
        print(f"✅ Saved with region_code → {output_path}")

    except Exception as e:
        print(f"❌ Failed for {nc_file}: {e}")

# === Save region_code map as JSON ===
with open(os.path.join(output_nc_folder, "region_code_map.json"), "w") as f:
    json.dump(reverse_map, f, indent=2)
    print("✅ Saved region_code_map.json")



📂 Processing: 2000.monthly_rain.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall\QLD_2000.monthly_rain.nc

📂 Processing: 2001.monthly_rain.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall\QLD_2001.monthly_rain.nc

📂 Processing: 2002.monthly_rain.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall\QLD_2002.monthly_rain.nc

📂 Processing: 2003.monthly_rain.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall\QLD_2003.monthly_rain.nc

📂 Processing: 2004.monthly_rain.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall\QLD_2004.monthly_rain.nc

📂 Processing: 2005.monthly_rain.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall\QLD_2005.monthly_rain.nc

📂 Processing: 2006.monthly_rain.nc
✅ Saved with region_code → Data - Modelling\Annual_DB\2. QLD_Cropped\Rainfall\QLD_2006.monthly_rain.nc

📂 Processing: 2007.monthly

Region wise variables Cropping Process

In [2]:
import os
import xarray as xr
import geopandas as gpd
import rioxarray

# === Paths ===
shapefile_path = r"Data\QLD Bio Regions\QLD_BioRegion_Shape_Files\Biogeographic_Regions.shp"
input_nc_folder = r"Data - Modelling\Annual_DB\1. AUS\Radiation"
output_nc_folder = r"Data - Modelling\Annual_DB\2. Reg_Cropped\Radiation"

os.makedirs(output_nc_folder, exist_ok=True)

# === Load shapefile ===
queensland_shape = gpd.read_file(shapefile_path)
queensland_shape = queensland_shape.to_crs("EPSG:4326")  # Match CRS to NetCDF

# === List all .nc files ===
nc_files = [f for f in os.listdir(input_nc_folder) if f.endswith(".nc")]

# === Loop through each NetCDF file ===
for nc_file in nc_files:
    nc_path = os.path.join(input_nc_folder, nc_file)
    print(f"\nProcessing file: {nc_file}")
    
    try:
        ds = xr.open_dataset(nc_path)
        ds = ds.rio.write_crs("EPSG:4326", inplace=True)

        # === Loop through each region ===
        for idx, row in queensland_shape.iterrows():
            region_code = row["q_reg"]
            region_name = row["q_reg_name"]
            geometry = [row.geometry]

            try:
                # Clip with region geometry
                clipped = ds.rio.clip(geometry, queensland_shape.crs, drop=True)
                
                # Add region info as metadata
                clipped.attrs["region_code"] = region_code
                clipped.attrs["region_name"] = region_name

                # Output filename
                output_filename = f"{region_code}_cropped_{nc_file}"
                output_path = os.path.join(output_nc_folder, output_filename)

                # Save to NetCDF
                clipped.to_netcdf(output_path)
                print(f"  ✅ Saved: {output_filename}")

            except Exception as region_err:
                print(f"  ⚠️  Skipped region {region_code}: {region_err}")
    
    except Exception as file_err:
        print(f"❌ Failed to process {nc_file}: {file_err}")



Processing file: 2000.radiation.nc
  ✅ Saved: BRB_cropped_2000.radiation.nc
  ✅ Saved: CHC_cropped_2000.radiation.nc
  ✅ Saved: CQC_cropped_2000.radiation.nc
  ✅ Saved: CYP_cropped_2000.radiation.nc
  ✅ Saved: DEU_cropped_2000.radiation.nc
  ✅ Saved: EIU_cropped_2000.radiation.nc
  ✅ Saved: GUP_cropped_2000.radiation.nc
  ✅ Saved: MGD_cropped_2000.radiation.nc
  ✅ Saved: MUL_cropped_2000.radiation.nc
  ✅ Saved: NET_cropped_2000.radiation.nc
  ✅ Saved: NWH_cropped_2000.radiation.nc
  ✅ Saved: SEQ_cropped_2000.radiation.nc
  ✅ Saved: WET_cropped_2000.radiation.nc

Processing file: 2001.radiation.nc
  ✅ Saved: BRB_cropped_2001.radiation.nc
  ✅ Saved: CHC_cropped_2001.radiation.nc
  ✅ Saved: CQC_cropped_2001.radiation.nc
  ✅ Saved: CYP_cropped_2001.radiation.nc
  ✅ Saved: DEU_cropped_2001.radiation.nc
  ✅ Saved: EIU_cropped_2001.radiation.nc
  ✅ Saved: GUP_cropped_2001.radiation.nc
  ✅ Saved: MGD_cropped_2001.radiation.nc
  ✅ Saved: MUL_cropped_2001.radiation.nc
  ✅ Saved: NET_cropped_200